In [1]:
# Exercise 8
!pip install tensorflow
from tensorflow.keras.datasets import mnist
mnist

2025-01-21 08:01:15.157037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737446475.209874    5868 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737446475.220000    5868 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 08:01:15.247620: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


<module 'keras.api.datasets.mnist' from '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/keras/api/datasets/mnist/__init__.py'>

In [2]:
from tensorflow.keras.datasets import mnist
import numpy as np

# Load the dataset
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Flatten the images from 28x28 to 784 dimensions
X_train_full = X_train_full.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

# Normalize the data (optional, but recommended for better training)
X_train_full = X_train_full.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Split the training data into 50,000 training and 10,000 validation sets
X_train, X_val = X_train_full[:50000], X_train_full[50000:]
y_train, y_val = y_train_full[:50000], y_train_full[50000:]

# Print the shapes to confirm
print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Validation labels shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}")


Training data shape: (50000, 784), Training labels shape: (50000,)
Validation data shape: (10000, 784), Validation labels shape: (10000,)
Test data shape: (10000, 784), Test labels shape: (10000,)


In [6]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC

rnd_clf = RandomForestClassifier(n_jobs=-1)
svm_clf = SVC(probability=True)
Xtrees_clf = ExtraTreesClassifier(n_jobs=-1)

voting_clf = VotingClassifier(
    estimators=[('rf', rnd_clf), ('Xtrees', Xtrees_clf)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(n_jobs=-1)),
                             ('Xtrees', ExtraTreesClassifier(n_jobs=-1))],
                 voting='soft')

In [7]:
from sklearn.metrics import accuracy_score

y_pred = voting_clf.predict(X_test)
testing_accuracy = accuracy_score(y_test, y_pred)
testing_accuracy

0.9728

In [8]:
rnd_clf.fit(X_train, y_train)
y_pred = rnd_clf.predict(X_test)
testing_accuracy = accuracy_score(y_test, y_pred)
testing_accuracy

0.9688

In [18]:
Xtrees_clf.fit(X_train, y_train)
#both algorithms are trained

#create the new training dataset
#rnd_clf_pred = rnd_clf.predict(X_val)
#Xtrees_clf_pred = Xtrees_clf.predict(X_val)

# Create the new dataset by stacking predictions
#X_new = np.column_stack([rnd_clf_pred, Xtrees_clf_pred])

estimators = [Xtrees_clf, rnd_clf]
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

for index, estimator in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

In [19]:
X_val_predictions

array([[3., 3.],
       [8., 8.],
       [6., 6.],
       ...,
       [5., 5.],
       [6., 6.],
       [8., 8.]], dtype=float32)

In [22]:
blender = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)
blender.fit(X_val_predictions, y_val)

#y_blender_pred = blender.predict(X_test)

#accuracy = accuracy_score(y_test, y_blender_pred)
#print(f"Blender Accuracy: {accuracy:.4f}")

RandomForestClassifier(n_estimators=200, n_jobs=-1, oob_score=True,
                       random_state=42)

In [23]:
blender.oob_score_

0.9731

In [24]:
# see how the blender performs on the test set
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

#get individual predictors to create a test set
for index, estimator in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

#train the blender using that test set
y_pred = blender.predict(X_test_predictions)
testing_accuracy = accuracy_score(y_test, y_pred)
testing_accuracy

0.9698

In [26]:
# add more classifiers
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier(n_jobs=-1)
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(n_jobs=-1)

In [27]:
#create the new training dataset
rnd_clf_pred = rnd_clf.predict(X_val)
Xtrees_clf_pred = Xtrees_clf.predict(X_val)
knn_clf_pred = knn_clf.predict(X_val)

# Create the new dataset by stacking predictions
X_new = np.column_stack([rnd_clf_pred, Xtrees_clf_pred, knn_clf_pred])

blender = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)
blender.fit(X_new, y_val)

RandomForestClassifier(n_estimators=200, n_jobs=-1, oob_score=True,
                       random_state=42)

In [29]:
rnd_clf_pred = rnd_clf.predict(X_test)
Xtrees_clf_pred = Xtrees_clf.predict(X_test)
knn_clf_pred = knn_clf.predict(X_test)

# Create the new dataset by stacking predictions
X_new = np.column_stack([rnd_clf_pred, Xtrees_clf_pred, knn_clf_pred])

y_pred = blender.predict(X_new)
testing_accuracy = accuracy_score(y_test, y_pred)
testing_accuracy

0.9709

In [30]:
testing_accuracy = accuracy_score(y_test, knn_clf_pred)
testing_accuracy

0.9664

In [31]:
testing_accuracy = accuracy_score(y_test, Xtrees_clf_pred)
testing_accuracy

0.9715

In [32]:
testing_accuracy = accuracy_score(y_test, rnd_clf_pred)
testing_accuracy

0.9688

In [36]:
# what if we try a grid search on the blender??? how much would it help
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [10, 20, 30],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],    # Minimum number of samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider at each split
}

grid_search = GridSearchCV(estimator=blender, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_new, y_val)

print("Best parameters found: ", grid_search.best_params_)

best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.7s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.0s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.5s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.7s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators

[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.5s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.6s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.7s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.6s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.7s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   1.9s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=50; total time=   0.6s
[CV] END max_depth=30, max_features=sqrt,

[CV] END max_depth=30, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   1.9s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, 

[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   1.7s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.5s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.4s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.7s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.8s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.5s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, max_features=log2

[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.5s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.6s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.5s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.4s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.7s
[CV] END max_depth=10, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.9s
[CV] END max_depth=10, max_features=log2, mi

...

[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.9s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.7s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.4s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.5s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.8s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.5s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   1.7s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.8s
[CV] END max_depth=20, max_features=sqrt,

[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, m

[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=50; total time=   0.0s
[CV] END max_depth=30, max_features=auto,

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
307 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py", line 666, in _val

Best parameters found:  {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
best_rf.fit(X_new, y_test)

rnd_clf_pred = rnd_clf.predict(X_val)
Xtrees_clf_pred = Xtrees_clf.predict(X_val)
knn_clf_pred = knn_clf.predict(X_val)

# Create the new dataset by stacking predictions
X_test_stacked = np.column_stack([rnd_clf_pred, Xtrees_clf_pred, knn_clf_pred])

y_pred = best_rf.predict(X_test_stacked)
testing_accuracy = accuracy_score(y_val, y_pred)
testing_accuracy